In [1]:
import pandas as pd
import numpy as np

In [2]:
#I saved the csv file in my folder locally; did not use a folders
#or .. syntax here for that reason. 
DataFrame = pd.read_csv("purchase_data.csv")

In [3]:
#view head of the data
DataFrame.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [4]:
#describe the data and create a copy just in case!
DataFrame.describe()
DataFrameCopy = DataFrame.copy()

In [5]:
#Player Count
DataFrame_playerscount = DataFrame["SN"].nunique()
print(f"There are {DataFrame_playerscount} unique players")

There are 576 unique players


In [7]:
#Purchasing Analysis (Total)
DataFrame_uniqueitems = DataFrame['Item Name'].nunique()
DataFrame_averageprice = DataFrame['Price'].mean()
NumberofPurchases = DataFrame['Purchase ID'].nunique()
TotalRevenue = DataFrame['Price'].sum()

SummaryPurchasing = pd.DataFrame({'Number of Unique Items': [DataFrame_uniqueitems],
                                  'Average Price': [DataFrame_averageprice],
                                  'Number of Purchases': [NumberofPurchases],
                                  'Total Revenue': [TotalRevenue]})
SummaryPurchasing

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,3.050987,780,2379.77


In [10]:
#Gender Demographics

#Percentage and count of male players
    
male_players = DataFrame.loc[DataFrame["Gender"] == "Male"]
female_players = DataFrame.loc[DataFrame["Gender"] == "Female"]
DataFrame.head()
other_players = DataFrame.loc[DataFrame["Gender"] == "Other / Non-Disclosed"]


male_count = male_players["SN"].nunique()
female_count = female_players["SN"].nunique()
total_players_count = DataFrame["SN"].nunique()
other_count = total_players_count - male_count - female_count

male_percentage = male_count / total_players_count
female_percentage = female_count / total_players_count
other_percentage = other_count / total_players_count


GenderDemographics = pd.DataFrame({"Gender Cat": ["Male","Female","Other"],
                                   "Percentage of Players": [male_percentage, female_percentage,other_percentage],
                                   "Total Counts": [male_count, female_count, other_count]})

GenderDemographics

,Gender Cat,Percentage of Players,Total Counts
0,Male,0.840278,484
1,Female,0.140625,81
2,Other,0.019097,11


In [14]:
#Age Demographics
    
age_bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

DataFrame["AgeBins"] = pd.cut(DataFrame["Age"], age_bins, labels = group_names)

AgePivot = pd.pivot_table(DataFrame, index = 'AgeBins',values = 'SN',
                          aggfunc = lambda x: len(x.unique()))  
AgePivot_2 = pd.pivot_table(DataFrame,index = 'AgeBins', values = 'SN',
                            aggfunc = 'count')

AgePivot_2["Percentage"] = (AgePivot_2["SN"]/AgePivot_2["SN"].count())

AgePivot_2.sort_values(by = 'AgeBins', ascending = True)



,SN,Percentage
AgeBins,,
<10,41,5.125
10-14,73,9.125
15-19,13,1.625
20-24,23,2.875
25-29,101,12.625
30-34,365,45.625
35-39,136,17.000
40+,28,3.500


In [17]:
#Purchasing Analysis by Age part 2
PurPivot = pd.pivot_table(DataFrame, values = ['SN','Price','Price','Price'])

PurchaseAgePivot_1 = pd.pivot_table(DataFrame, values = ['SN'], index = ['AgeBins'],
                     aggfunc = [np.count_nonzero])

PurchaseAgePivot_2 = pd.pivot_table(DataFrame, values = ['Price'], index = ['AgeBins'],
                     aggfunc = [np.average])

PurchaseAgePivot_3 = pd.pivot_table(DataFrame, values = ['Price'], index = ['AgeBins'],
                     aggfunc = [np.sum])


PurchaseAgePivot_4 = pd.pivot_table(DataFrame, values = ['Price'], index = ['AgeBins'],
                     aggfunc = [np.average])


Purch_Pivot_Combined = pd.concat((PurchaseAgePivot_1, PurchaseAgePivot_2,PurchaseAgePivot_3,PurchaseAgePivot_4), axis=1)
Purch_Pivot_Combined.sort_values(by = 'AgeBins', ascending = True)

ValueError: No group keys passed!

In [22]:
#Top Spenders

TopGrouped1 = pd.DataFrame(DataFrame.groupby('SN')['Purchase ID'].nunique())
TopGrouped2 = pd.DataFrame(DataFrame.groupby('SN')['Price'].mean())
TopGrouped3 = pd.DataFrame(DataFrame.groupby('SN')['Price'].sum())
TopGrouped_Combined = pd.concat((TopGrouped1,TopGrouped2,TopGrouped3),axis = 1)
TopGrouped_Combined_Top5 = TopGrouped_Combined.nlargest(5,'Purchase ID')
TopGroped_Combined_Top5

NameError: name 'TopGroped_Combined_Top5' is not defined

In [25]:
#Most Popular Items
TopGrouped4 = pd.DataFrame(DataFrame.groupby(['Item ID'])['Purchase ID'].nunique())
TopGrouped4.columns = ['Count of Purchases']
TopGrouped5 = pd.DataFrame(DataFrame.groupby(['Item ID'])['Price'].mean())
TopGrouped5.columns = ['PriceAvg']
TopGrouped6 = pd.DataFrame(DataFrame.groupby(['Item ID'])['Price'].sum())
TopGrouped6.columns = ['Price Sum']

TopGrouped_Combined_Items = pd.concat((TopGrouped4,TopGrouped5,TopGrouped6),axis = 1)

TopGrouped_Combined_Items_Top5 = TopGrouped_Combined_Items.nlargest(5,'Count of Purchases')
TopGrouped_Combined_Items_Top5

,Count of Purchases,PriceAvg,Price Sum
Item ID,,,
178,12,4.23,50.76
82,9,4.90,44.10
108,9,3.53,31.77
145,9,4.58,41.22
19,8,1.02,8.16


In [27]:
#Most Profitable Items
TopGrouped_Combined_Items_Profitable = TopGrouped_Combined_Items.nlargest(5,'Price Sum')
TopGrouped_Combined_Items_Profitable

,Count of Purchases,PriceAvg,Price Sum
Item ID,,,
178,12,4.23,50.76
82,9,4.90,44.10
145,9,4.58,41.22
92,8,4.88,39.04
103,8,4.35,34.80
